<h1>Part 1</h1>

First let us import all the packages and libraries we will be using in this project.

In [1]:
import pandas as pd
import requests

In [ ]:
pip install lxml

In [3]:
import lxml

Now, we will load the data from the given Wikipedia page by using get method and then use read_html which stores all the tables from html page to a list of dataframes.

In [4]:
toronto_data=pd.read_html(requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content)

As we got our list of dataframes, we will select the dataframe required for our project. Here we require the dataframe stored at index 0.  

In [5]:
toronto_data=toronto_data[0]

In [6]:
toronto_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Now, we will drop all the rows where Borough==Not assigned and then reseting our index values.

In [7]:
df=toronto_data['Borough']!='Not assigned'
df

0      False
1      False
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178     True
179    False
Name: Borough, Length: 180, dtype: bool

In [8]:
toronto_data=toronto_data[df]
toronto_data=toronto_data.reset_index(drop=True)
toronto_data.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"



Now, let's check the shape of our dataframe.

In [9]:
toronto_data.shape

(103, 3)

<h1>Part 2</h1>

In this part we have to extract the coordinates for each neighbourhood. For this we have two methods: 1.through python geocoder package and 2.Get the already stored coordinates from csv file.
You can use the code given below to extract the latitude and longitude of neighbourhoods using geocoder. But here we will use the csv file to get our coordinates and will try to store the coordinates into our dataframe i.e. <b>toronto_data</b> so that we can use these coordinates to access Foursquare API.

<p>!conda install -c conda-forge geocoder --yes 
   
import geocoder
<pre>
for postal,borough,neigh in zip(toronto_data['Postal Code'],toronto_data['Borough'],toronto_data['Neighbourhood']):
    lat_lng_coords = None<br>
    while(lat_lng_coords is None):
            g = geocoder.google('M3A,North York,Parkwoods')
            lat_lng_coords = g.latlng
            
    toronto_data['latitude'] = lat_lng_coords[0]
    toronto_data['longitude'] = lat_lng_coords[1]
    </pre></p>
    

In [10]:
file=pd.read_csv("Geospatial_Coordinates.csv")

In [11]:
file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
toronto_data=pd.merge(toronto_data,file,how='inner',on='Postal Code')

In [17]:
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
